In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
np.set_printoptions(edgeitems=15,linewidth=600)

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.mip_model import *

In [2]:
start = time.time()

In [39]:
data = pd.read_excel('../data/data_template_blank.xlsx').head(2)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)

In [40]:
data

,vehicle_idx,equipmentid,vehicledescription,vehicle_type,dept_name,deptid,purchasedate,purchaseprice,depreceated value,miles2020,...,maintenance2020,cumulative_maintenance,mpg2020,emissions2020,replacementvehicle_id,replacement_vehicle_desc,replacement_vehicle_purchaseprice,replacement_vehicle_mpge,replacement_vehicle_type,current_age
0,0,24399,1999 GMC 3/4 TON PU',ICE,DPW: Solid Waste Management (Refuse Disposal),10707603,1999-03-01,23886.0,3000.0,4714,...,3512.53,23036.5,13.0,877.891846,NaN,Ford F 150 XLT Super Crew,39000.0,30.6,HEV,22
1,1,26142,2002 FORD VAN CARGO',ICE,Baltimore County Department of Library,BS001-2180,2002-04-09,21913.0,7000.0,3641,...,46.82,6064.33,0.0,NaN,NaN,Ford Transit 350HD Cargo Van,46000.0,66.0,EV,19


In [41]:
mip = MIP(data)

In [42]:
keepSchedules = mip.make_keep_schedules()

In [43]:
age = mip.get_vehicle_age(keepSchedules)#[50:80]

In [44]:
annual_mileage = mip.make_mileage()

In [45]:
odometer = mip.make_odometer(annual_mileage,age)

In [46]:
acquisition = mip.get_acquisition_cost()

In [47]:
is_ice,is_ev = mip.get_vehicle_type_trackers()

In [51]:
next vv

SyntaxError: invalid syntax (<ipython-input-51-2373e5fb8a88>, line 1)

In [48]:
consumables = mip.get_consumables(annual_mileage,is_ice,is_ev)

In [49]:
maintenance = mip.get_maintenance_cost(age,annual_mileage,odometer)

In [50]:
emissions = mip.get_emissions(annual_mileage,is_ice,is_ev)

In [38]:
infeasible_filter = mip.find_infeasible_schedules(odometer,age)
# mip.replacement_schedules[~infeasible_filter]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       ...,
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1,

In [ ]:
vehicles = [v for v in range(0,num_vehicles)]
schedules = [s for s in range(0,num_schedules)]
years = [t for t in range(0,num_years)]
finalYear = max(years)

In [ ]:
c = {}
a = {}
m = {}
e = {}
for v in vehicles:
    for s in schedules:
        if not infeasible_filter[v,s]:
            c[v,s] = consumables[v,s]
            a[v,s] = acquisition[v,s]
            m[v,s] = maintenance[v,s]
            e[v,s] = emissions[v,s]

In [ ]:
consumables = c.copy()
acquisition = a.copy()
maintenance = m.copy()
emissions = e.copy()

In [ ]:
#! I think I can pull the code I used in modelinputsgen
budget_acquisition = 1300000*np.ones(shape=(15)) 
budget_operations = 1000000*np.ones(shape=(15))

emissions_goal = 40000

numDesiredSolutions = 3

In [ ]:
try: 
    m.reset()
    del m    
except:
    None
    
m = grb.Model('carnet')

In [ ]:
m.setParam('PoolSearchMode',2) #tell gurobi I want multiple solutions
m.setParam('PoolSolutions',numDesiredSolutions) #number of solutions I want
m.setParam('TimeLimit',30)

In [ ]:
x = m.addVars(vehicles,schedules,vtype=grb.GRB.BINARY,name='x')
penalty_budget = m.addVar(vtype=grb.GRB.CONTINUOUS,name='penalty_budget')
penalty_emissions = m.addVar(vtype=grb.GRB.CONTINUOUS,name='penalty_emissions')
# y = m.addVars(years,)

In [ ]:
w = {'cost':0.70,'emissions':0.30}

In [ ]:
# total_cost = np.sum(consumables+acquisition+maintenance,axis=2)
# total_acquisition_cost = np.sum(acquisition,axis=2)
# total_operations_cost = np.sum(consumables+maintenance,axis=2)

In [ ]:
validSchedules = list(consumables.keys())

In [ ]:
obj = m.setObjective(grb.quicksum(w['cost']*consumables[v,s][t]*x[v,s] for v,s in validSchedules for t in years) + 
                     grb.quicksum(w['emissions']*emissions[v,s][finalYear]*x[v,s] for v,s in validSchedules) +
                     1000000*(penalty_budget+penalty_emissions),grb.GRB.MINIMIZE)
# obj = m.setObjective(grb.quicksum(w['cost']*(consumables[v,s,t]+acquisition[v,s,t]+maintenance[v,s,t])*x[v,s] + 
#                                   w['emissions']*emissions[v,s,finalYear]*x[v,s] for v in vehicles for s in schedules for t in years),grb.GRB.MINIMIZE)

In [ ]:
import pandas as pd
validSchedulesPerVehicle = pd.DataFrame(validSchedules).groupby(0)[1].unique().to_dict()

In [ ]:
c1 = m.addConstrs((grb.quicksum(x[v,s] for s in validSchedulesPerVehicle[v])==1 for v in vehicles),'one_schedule_per_vehicle')

In [ ]:
c2 = m.addConstrs((grb.quicksum((consumables[v,s][t]+maintenance[v,s][t])*x[v,s] for v,s in validSchedules) <= budget_operations[t]+penalty_budget for t in years),'operations_budget')
c3 = m.addConstrs((grb.quicksum(acquisition[v,s][t]*x[v,s] for v,s in validSchedules) <= budget_operations[t]+penalty_budget for t in years),'acquisition_budget')

In [ ]:
c4 = m.addConstr((grb.quicksum(emissions[v,s][finalYear]*x[v,s] for v,s in validSchedules) <= emissions_goal+penalty_emissions),'emissions_goal')

In [ ]:
# c5 = m.addConstrs((infeasible_filter[v,s]*x[v,s] <= 0 for v in vehicles for s in schedules),'infeasible_schedules')

In [ ]:
m.optimize()

In [ ]:
end = time.time()
print("--- %s seconds ---" % (time.time() - start))


In [ ]:
#multiple solutions
solution = []
options = ['A','B','C']
for solution in range(0,299):
    print()
    print(solution)
#     print(f'Option: {options[solution]}')
    m.setParam('SolutionNumber',solution)
    for v in vehicles:
        for s in schedules:
            if x[v,s].xn==1:
                print(f'   Vehicle: {v+1} Schedule: {s} {replacementSchedules[v,s]}')     

In [ ]:
print(f'age: {age[0,29]}')
print()
print(f'annual_mileage: {annual_mileage[0,29]}')
print()
print(f'odometer: {odometer[0,29]}')
print()
print(f'acquisition cost: {acquisition[0,29]}')
print()
print(f'maintenance cost: {maintenance[0,29]}')
print()
print(f'consumables cost: {consumables[0,29]}')
print()
print(f'emissions: {emissions[0,29]}')